# Background and Problem Space
## Background
I am looking to find some probabalistic model to determine likely duplicate strings. The strings have a common grammer and structure, additionally assertained from similar texts. 

For an example that we will use for the remainder of the task our strings are of the form "Does the privacy policy affirm {X}?", where `X` is some variable statement. And they're all assertained from different privacy policies about similar products.

I have then for each string in our corpus a selection of embedding vectors, each embedding vector is assertained from the same fundamental model, however there are slight differences. Some have been assertained by manipulating the string, e.g via these two example functions (written in python as a demonstration):
```
def example_question_manipulation_1(question):
	prefix = "Does the privacy policy affirm"
	replacement = "The privacy policy affirms"
	processed_text = ("The privacy policy affirms" + question[len(prefix) :])[:-1] + "."

	return processed_text

def example_question_manipulation_2(question):
	prefix = "Does the privacy policy affirm that "
	processed_text = (question[len(prefix) :].capitalize())[:-1] + "."
	return processed_text
```
And the specific task type e.g "SEMANTIC_SIMILARITY", "FACT_VERIFICATION" etc, we will use `gemini-embedding-001` as our cannonical embedding model.
## Problem Space
We want to derive the factors required to construct a probabilistic model $P(\text{Duplicate} \mid X)$, where $X$ is a multivariate state including:
-   The distance vector $\vec{d}$ (Mahalanobis, Cosine, etc.).
-   The embedding source type (e.g., specific manipulation functions).
-   Consensus signals (whether the pair is a nearest neighbor across multiple embedding views, whether we have highly overlapping clustors).
-   Local topology of clusters and vector spaces
-   Similarities in topology of vector spaces, since we expect different embedding types to be correlated since they all represent "views" on the same fundamental information.

**Constraints & Assumptions:**
1. **Zero-Shot / Unsupervised:** We have no labeled 'true duplicates'.
2. **No Arbitrary Heuristics:** We reject metrics like "top 1% nearest neighbors." All probabilities must be inferred from the structural properties of the vector space and the consensus between embedding views.
3. **Multivariate Dependencies:** The probability is not solely a function of scalar distance. It may be conditioned on the specific embedding model used, the stability of the nearest-neighbor relationship across models, etc.

## Tentative Considerations for the Solution Space
I have identified the preliminarr empirical signals (metrics). Please analyze how these function as variables in a probabilistic framework (e.g., as priors, likelihood ratios, or density estimation parameters):

1.  **Metric: Pairwise Jaccard Index**
    The intersection over union of pair sets identified by different embedding models.
2.  **Metric: Average Consensus-Supporting Group Count (ACGC):**
	The average count of clusters per model that are validated by the intersection of all pair sets.
3.  **Factor: Empirical distributions of Nearest Neighbours**
4.  **Factor:Vector & Distance Deltas**
    The raw distance metrics ($D$) and the variance of $D$ across different embedding manipulations.
5.  **Idea: Pairwise filtering:**
	Filter pairs for our metrics based on some conditions, i.e using all pairs in some cluster, only using nearest neighbour pairs. This may help us determine the relationship between duplicates, being a nearest neighbour vs not being a nearest neighbour

**Important:**
In the following task is about defining the problem space, metrics, distributions and assumptions **NOT** creating the duplicate detection model.


# Task
I would like you to help me fill out relevant factors and metrics we can use in our model.

Please determine any other potentially relevant things to consider that may be valuable to our model,

The initial start that I have determined in our solution space is as following.
- Quantify the distance delta between two embedding vectors, using some distance metric D (e.g Mahalanobis, Cosine, L1, L2)
- We can determine some useful empirical distributions and utilise (Extreme Value Theory) and other mathematical frameworks, e.g distributions of nearest neighbours.
- We can then cluster distances via AgglomerativeClustering
- We can threshold distances from Agglomerative Clustering, this can give us some other useful metrics like:
	- **Threshold Minimisations**:
		- Maximise the Jaccard Index of pairs between embedding_vector types: `len(set.intersection(*pair_sets))/set.union(*pair_sets)`
		- Maximise our Average Consensus-Supporting Group Count (ACGC) i.e the average count of clusters per model that are validated by the intersection of all pair sets.
	- **Threshold Maximisation**:
		- Given a metric score find the maximum distance threshold that retains this index, or more specific constraints e.g:
		For `Jaccard Index` we can maximize distance threshold T, subject to the constraint that the set of identified duplicate pairs remains invariant

We can also for these thresholding restrict our pairsets, e.g from pairsets in a cluster, to pairsets in a cluster which are nearest neighbours. For some metrics like Jaccard our score may be similar (I may even suggest that it will with high likelihood be identical), or minimialy change (ACGC). This may be interesting since our empirical distributions assume that all semantic duplicates will be a nearest neighbour in at least one model. However our metrics if we chose all pairs in clusters include non-nearest neighbour pairs. We may be able to infer some likelihood of (duplicates | not nearest neighbour) from manipulation of our pairing mechanism.

With well-behaved vector spaces and with some assumptions we may be able to compute some other metrics and distributions, i.e:
- Deduce correlation of vectors associated with the same string given different embedding vector types.
- Deduce correlation of distances associated with the same pairing of given different embedding vector types.

# Constraints:
We are not solving the actual problem yet, we are only expanding the factors in our solution space, therefore I expect:
- Mainly mathematical derivations and qualitiative discussions. I expect you will only return codeblocks if absolutely necessary to demonstrate some example or test.
- If I ask specific questions, your answer should be targetted and precise. Treat each question as self contained, and the answer does not modify the context of our discussion and solution space until I explicitly say we will add it to. No premature implementations/incorperations before we have fully discussed the answer.
- Treat this as a Zero-Shot Unsupervised problem. We have no labeled duplicates. The definition of a 'duplicate' must be inferred from the structural properties of the vector space and the consensus between embedding views."
- We strictly reject the use of arbitrary, exogenous thresholds (e.g., "fixed distance < 0.5" or "top 1% nearest neighbors") as they fail to capture the underlying uncertainty of the data.


In [ ]:
import os
import json

QUESTIONS_FILE = "./data/questions_filter_after.json"


def _loadJson(filepath):
	if not os.path.exists(filepath):
		print(f"Warning: File not found: {filepath}")
		return {}
	try:
		with open(filepath, "r", encoding="utf-8") as f:
			return json.load(f)
	except json.JSONDecodeError:
		print(f"Error decoding JSON: {filepath}")
		return {}


qdata = _loadJson(QUESTIONS_FILE)

In [23]:
import numpy as np

In [24]:
# executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_policy_chunks)
# e_dict = dict()
# for index, chunk in enumerate(policy_chunks):
# 	e_dict[index] = executor.submit(
# 		self.processPolicyChunks, chunk, policy_hash, policy_name, index
# 	)
# executor.shutdown(wait=True)

In [25]:
import concurrent.futures

In [26]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.covariance import LedoitWolf


# abstracted
def getMahalanobisDistances(vectors_a, vectors_b):
	norms = np.linalg.norm(vectors_a, axis=1, keepdims=True)
	cleaned_vectors = vectors_a / (norms + 1e-10)

	lw = LedoitWolf()
	lw.fit(cleaned_vectors)	# assumption vectors_a=vectors_b
	precision_matrix = lw.precision_

	dist_matrix = cdist(
		cleaned_vectors, cleaned_vectors, metric="mahalanobis", VI=precision_matrix
	)
	return dist_matrix, precision_matrix


Distance_Processors = {
	"cosine": lambda emb_a, emb_b: 1.0
	- (emb_a @ emb_b.T)
	/ (
		np.linalg.norm(emb_a, axis=1, keepdims=True)
		@ np.linalg.norm(emb_b, axis=1, keepdims=True).T
		+ 1e-10
	),
	"l1": lambda emb_a, emb_b: np.sum(np.abs(emb_a[..., np.newaxis] - emb_b.T), axis=1),
	"l2": lambda emb_a, emb_b: np.linalg.norm(emb_a[..., np.newaxis] - emb_b.T, axis=1),
	"dot": lambda emb_a, emb_b: emb_a @ emb_b.T,
	"mahalanobis": lambda emb_a, emb_b: getMahalanobisDistances(emb_a, emb_b),
}


def _prepareModelArtifact(
	raw_vectors,
	semantic_data,
	truncation_dim=256,
	distance_metric="mahalanobis",
	debug=True,
):
	# 1. Truncation
	data_matrix = np.array(raw_vectors)
	input_dim = data_matrix.shape[1]

	if input_dim < truncation_dim and debug:
		print(
			f"Warning: Vector dimension ({input_dim}) is smaller than truncation limit ({truncation_dim}). Proceeding without truncation."
		)

	data_truncated = data_matrix[:, :truncation_dim]

	# 2. Distance Calculation
	dist_output = Distance_Processors[distance_metric](data_truncated, data_truncated)

	precision_matrix = None
	if distance_metric == "mahalanobis":
		dist_matrix, precision_matrix = dist_output
	else:
		dist_matrix = dist_output

	# 3. NN Indices (In-place modification to avoid copy overhead)
	np.fill_diagonal(dist_matrix, float("inf"))
	nn_indices = np.argmin(dist_matrix, axis=1)
	np.fill_diagonal(dist_matrix, 0.0)

	return {
		"dist_matrix": dist_matrix,
		"vectors": data_truncated,
		"precision": precision_matrix,
		"semantic_data": semantic_data,
		"metric": distance_metric,
		"nn_indices": nn_indices,
	}


def prepareModelArtifacts(
	data_set, vector_keys, truncation_dim=256, distance_metric="mahalanobis", debug=True
):
	semantic_data = list(data_set.keys())
	model_artifacts = {}
	raw_vectors = {}
	for key in vector_keys:
		raw_vectors[key] = [data_set[s][key] for s in semantic_data]

	for key in vector_keys:
		if debug:
			print(f"Processing {key}...")

		model_artifacts[key] = _prepareModelArtifact(
			raw_vectors[key],
			semantic_data,
			truncation_dim,
			distance_metric,
			debug,
		)

	return model_artifacts

In [27]:
import numpy as np
from scipy.stats import rankdata


# def computeLocalDensityRanks(dist_matrix, k_neighbors=50):
# 	"""
# 	Converts raw scalar distances into Local Density Scores (Empirical CDF).

# 	This effectively normalizes the manifold: a distance of 0.2 is treated
# 	differently in a dense cluster vs. a sparse region.

# 	Args:
# 	    dist_matrix: (N x N) Raw distance matrix (e.g., Mahalanobis).
# 	    k_neighbors: The effective 'local bandwidth'. We normalize ranks
# 	                 relative to this count.

# 	Returns:
# 	    local_rank_matrix: (N x N) Values in [0, 1].
# 	                       Values near 0 indicate the pair is within the
# 	                       tightest local neighbors. Values clipped at 1.0
# 	                       indicate 'Background/Noise'.
# 	"""
# 	# 1. Compute Ranks Row-wise
# 	# method='average' assigns the average rank to ties, preserving statistical properties
# 	# We rank strictly along axis 1 (neighbors of row i)
# 	ranks = rankdata(dist_matrix, axis=1, method="average")

# 	# 2. Normalize by k_neighbors (The Local Density assumption)
# 	# If k=50, the 1st neighbor gets score 0.02, the 50th gets 1.0.
# 	# The 51st neighbor gets 1.02.
# 	normalized_ranks = ranks / k_neighbors

# 	# 3. Clip to [0, 1]
# 	# We treat anything outside the local neighborhood (rank > k) as
# 	# effectively "Probability 1.0" (Maximum Entropy / Background).
# 	# In EVT terms, we only care about the tail approaching 0.
# 	local_rank_matrix = np.clip(normalized_ranks, 0.0, 1.0)

# 	return local_rank_matrix


def computeLocalDensityRanks(dist_matrix, k_neighbors=50):
	"""
	Revised: Subtracts 1 from rank so Self-Match is 0.0 and
	First Neighbor is 1.0/k.
	"""
	# method='min' ensures ties get the lower rank (good for strict duplicates)
	ranks = rankdata(dist_matrix, axis=1, method="min")

	# Subtract 1 so the diagonal (self) is 0.0
	# The first NN is now 1.0
	ranks = ranks - 1.0

	# Normalize
	normalized_ranks = ranks / k_neighbors

	# Clip [0, 1]
	local_rank_matrix = np.clip(normalized_ranks, 0.0, 1.0)

	return local_rank_matrix


def computeFusedEVTStatistic(rank_matrices_dict):
	"""
	Combines views using the Extreme Value Theory 'Minimum' assumption.

	Hypothesis:
	    If two strings are equivalent, they will collide (rank -> 0) in
	    AT LEAST ONE valid semantic manipulation.

	Args:
	    rank_matrices_dict: Dictionary {model_key: (N x N) local_rank_matrix}.

	Returns:
	    fused_statistic_matrix: (N x N) The observation variable X.
	                            Represents the "best case" topological
	                            closeness across all views.
	"""
	if not rank_matrices_dict:
		raise ValueError("rank_matrices_dict is empty.")

	# Convert dict values to a list of matrices
	matrices = list(rank_matrices_dict.values())

	# Stack along a new axis (M, N, N)
	stacked_matrices = np.stack(matrices, axis=0)

	# Compute element-wise minimum along the model axis
	fused_statistic_matrix = np.min(stacked_matrices, axis=0)

	return fused_statistic_matrix

In [28]:
# # --- Test Data Generation ---
# # Model A: Standard sorted distances
# dist_A = np.array(
# 	[
# 		[0.0, 0.1, 0.2, 0.3, 0.4],
# 		[0.1, 0.0, 0.5, 0.5, 0.8],	# Note the tie at 0.5
# 		[0.2, 0.5, 0.0, 0.9, 0.1],
# 		[0.3, 0.5, 0.9, 0.0, 0.2],
# 		[0.4, 0.8, 0.1, 0.2, 0.0],
# 	]
# )

# # Model B: Inverted distances (to test Fusion)
# # e.g., what was far in A is close in B
# dist_B = np.array(
# 	[
# 		[0.0, 0.4, 0.3, 0.2, 0.1],
# 		[0.4, 0.0, 0.2, 0.1, 0.3],
# 		[0.3, 0.2, 0.0, 0.1, 0.5],
# 		[0.2, 0.1, 0.1, 0.0, 0.4],
# 		[0.1, 0.3, 0.5, 0.4, 0.0],
# 	]
# )

# print("--- Test: Local Density Ranks (k=5) ---")
# # Using k=5 means ranks 1-5 become probabilities 0.2, 0.4, 0.6, 0.8, 1.0
# rank_A = computeLocalDensityRanks(dist_A, k_neighbors=5)
# rank_B = computeLocalDensityRanks(dist_B, k_neighbors=5)

# print("\nMatrix A Ranks (Row 0 should be ordered 0.2 -> 1.0):")
# print(rank_A[0])

# print("\nMatrix A Row 1 (Handling Ties at 0.5):")
# # Distances: [0.1, 0.0, 0.5, 0.5, 0.8]
# # Ranks:     [2,   1,   3.5, 3.5, 5  ] (Average rank for ties)
# # Norm:      [0.4, 0.2, 0.7, 0.7, 1.0]
# print(f"Distances: {dist_A[1]}")
# print(f"Ranks:     {rank_A[1]}")

# print("\n--- Test: EVT Fusion (Minimum) ---")
# rank_dict = {"Model_A": rank_A, "Model_B": rank_B}
# fused = computeFusedEVTStatistic(rank_dict)

# print("Fused Matrix Row 0:")
# print(f"Rank A: {rank_A[0]}")
# print(f"Rank B: {rank_B[0]}")
# print(f"Fused : {fused[0]}")

# # Verification for Row 0, Index 1:
# # A=0.4 (Dist 0.1), B=1.0 (Dist 0.4). Min should be 0.4.
# assert fused[0, 1] == 0.4
# print("\nTest passed successfully.")

In [29]:
import numpy as np
from scipy.stats import rankdata

# # --- REFINED HELPER FUNCTIONS ---


# def computeLocalDensityRanks(dist_matrix, k_neighbors=50):
# 	"""
# 	Revised: Subtracts 1 from rank so Self-Match is 0.0 and
# 	First Neighbor is 1.0/k.
# 	"""
# 	# method='min' ensures ties get the lower rank (good for strict duplicates)
# 	ranks = rankdata(dist_matrix, axis=1, method="min")

# 	# Subtract 1 so the diagonal (self) is 0.0
# 	# The first NN is now 1.0
# 	ranks = ranks - 1.0

# 	# Normalize
# 	normalized_ranks = ranks / k_neighbors

# 	# Clip [0, 1]
# 	local_rank_matrix = np.clip(normalized_ranks, 0.0, 1.0)

# 	return local_rank_matrix


# def _initializeParametersViaConsensus(
# 	fused_statistic_matrix, percentile_threshold=0.02
# ):
# 	"""
# 	Relaxed threshold to 0.02 (2%) to ensure we catch the signal tail
# 	before the EM algorithm refines it.
# 	"""
# 	return _initializeParametersViaNaiveQuantile(
# 		fused_statistic_matrix, signal_quantile=percentile_threshold
# 	)


# --- RE-RUN PIPELINE ---

print(">>> Preparing Model Artifacts...")
artifacts = prepareModelArtifacts(qdata, model_keys, truncation_dim=256, debug=False)

print("\n>>> Running Probabilistic Pipeline (Debug Mode)...")

# 1. Local Density
rank_matrices = {}
for key, artifact in artifacts.items():
	rank_matrices[key] = computeLocalDensityRanks(artifact["dist_matrix"], k_neighbors=50)

# 2. Fusion
fused_matrix = computeFusedEVTStatistic(rank_matrices)

# DEBUG: Check the data range
off_diag = fused_matrix[~np.eye(fused_matrix.shape[0], dtype=bool)]
print(
	f"Fused Data Stats: Min={off_diag.min():.4f}, Max={off_diag.max():.4f}, Mean={off_diag.mean():.4f}"
)
print(f"Values < 0.05: {np.sum(off_diag < 0.05)} pairs")

# 3. Initialization
# Using 2% quantile
init_params = _initializeParametersViaConsensus(fused_matrix, percentile_threshold=0.02)
print(f"New Init Params: {init_params}")

# 4. Modeling
likelihood_matrix, final_params = fitWeibullBetaMixture(
	fused_matrix, initial_params=init_params
)
print(f"Final Model Params: {final_params}")

# 5. Topology
prior_matrix = computeTransitivityPrior(likelihood_matrix)
posterior_matrix = computeFinalPosterior(likelihood_matrix, prior_matrix)

# 6. Extract
extractAndPrintClusters(
	posterior_matrix, artifacts[model_keys[0]]["semantic_data"], threshold=0.5
)

>>> Preparing Model Artifacts...


KeyboardInterrupt: 

In [ ]:
import numpy as np
from scipy.stats import expon, beta
from sklearn.mixture import GaussianMixture


def _get_off_diagonal_samples(matrix):
	"""Helper: Flattens matrix and removes self-matching diagonal."""
	N = matrix.shape[0]
	# Create mask for off-diagonal
	mask = ~np.eye(N, dtype=bool)
	return matrix[mask]


def _initializeParametersViaNaiveQuantile(fused_statistic_matrix, signal_quantile=0.01):
	"""
	Fallback initialization.
	Assumes bottom 1% are Signal (Exponential), top 99% are Noise (Beta).
	"""
	data = _get_off_diagonal_samples(fused_statistic_matrix)

	# 1. Hard split
	threshold = np.quantile(data, signal_quantile)
	signal_data = data[data <= threshold]
	noise_data = data[data > threshold]

	# 2. Estimate Signal (Exponential: lambda = 1 / mean)
	# Add epsilon to avoid divide by zero if perfect match
	lamb = 1.0 / (np.mean(signal_data) + 1e-6)

	# 3. Estimate Noise (Beta: Method of Moments)
	mu = np.mean(noise_data)
	var = np.var(noise_data)

	# Beta Method of Moments constraints
	if var >= mu * (1 - mu):
		# Fallback if variance is too high for Beta (implies bimodal/uniform)
		alpha, b_param = 1.0, 1.0
	else:
		common = (mu * (1 - mu) / (var + 1e-9)) - 1
		alpha = mu * common
		b_param = (1 - mu) * common

	return {
		"pi": signal_quantile,
		"lambda": lamb,
		"alpha": max(alpha, 1.0),	# Enforce > 1 for bell/flat shapes
		"beta": max(b_param, 1.0),
	}


def _initializeParametersViaConsensus(
	fused_statistic_matrix, percentile_threshold=0.02
):
	"""
	Relaxed threshold to 0.02 (2%) to ensure we catch the signal tail
	before the EM algorithm refines it.
	"""
	return _initializeParametersViaNaiveQuantile(
		fused_statistic_matrix, signal_quantile=percentile_threshold
	)


# def _initializeParametersViaConsensus(
# 	fused_statistic_matrix, percentile_threshold=0.01
# ):
# 	"""
# 	Bootstraps using a 'High-Precision' assumption.
# 	Since we don't have the Jaccard sets here, we assume 'Consensus'
# 	refers to the extremely low-rank tail (e.g., top 0.1%).
# 	This is a stricter version of the Naive Quantile.
# 	"""
# 	# Use a much stricter quantile to simulate "Consensus Intersection"
# 	return _initializeParametersViaNaiveQuantile(
# 		fused_statistic_matrix, signal_quantile=0.001
# 	)


def fitWeibullBetaMixture(
	fused_statistic_matrix, initial_params=None, max_iter=20, tol=1e-4
):
	"""
	Fits: P(x) = pi * Exp(x|lambda) + (1-pi) * Beta(x|alpha, beta)
	Using a simplified EM algorithm.
	"""
	data = _get_off_diagonal_samples(fused_statistic_matrix)
	# Clip data to (epsilon, 1-epsilon) for numerical stability of Beta/Log
	data = np.clip(data, 1e-6, 1.0 - 1e-6)

	# --- Initialization ---
	if initial_params is None:
		params = _initializeParametersViaNaiveQuantile(fused_statistic_matrix)
	else:
		params = initial_params

	pi = params["pi"]
	lamb = params["lambda"]
	a = params["alpha"]
	b = params["beta"]

	log_likelihood_old = -np.inf

	# --- EM Loop ---
	for i in range(max_iter):
		# 1. E-Step: Calculate Responsibilities (Posterior of Hidden Var Z)

		# PDF of Signal (Exponential)
		pdf_signal = expon.pdf(data, scale=1.0 / lamb)

		# PDF of Noise (Beta)
		pdf_noise = beta.pdf(data, a, b)

		# Weighted Probs
		weighted_signal = pi * pdf_signal
		weighted_noise = (1 - pi) * pdf_noise

		# Normalization (Total Evidence)
		total_evidence = weighted_signal + weighted_noise + 1e-10

		# Gamma: P(Z=Signal | x)
		gamma = weighted_signal / total_evidence

		# 2. M-Step: Update Parameters

		# N_signal (Effective count of signal items)
		N_s = np.sum(gamma)
		N_n = len(data) - N_s

		# Update Pi (Mixing Coefficient)
		pi = N_s / len(data)

		# Update Lambda (MLE for Weighted Exponential = 1 / weighted_mean)
		weighted_sum_x = np.sum(gamma * data)
		lamb = 1.0 / (weighted_sum_x / (N_s + 1e-10))

		# Update Alpha/Beta (Weighted Method of Moments for speed/stability)
		# Calculate weighted mean/var for the Noise component (1-gamma)
		w_noise = 1 - gamma
		w_noise_sum = np.sum(w_noise) + 1e-10

		mu_n = np.sum(w_noise * data) / w_noise_sum
		var_n = np.sum(w_noise * (data - mu_n) ** 2) / w_noise_sum

		if var_n < mu_n * (1 - mu_n):
			common = (mu_n * (1 - mu_n) / (var_n + 1e-10)) - 1
			a = max(mu_n * common, 1.0)
			b = max((1 - mu_n) * common, 1.0)

		# Check convergence
		log_likelihood_new = np.sum(np.log(total_evidence))
		if abs(log_likelihood_new - log_likelihood_old) < tol:
			break
		log_likelihood_old = log_likelihood_new

	final_params = {"pi": pi, "lambda": lamb, "alpha": a, "beta": b}

	# --- Compute Final Posterior Matrix ---
	# Apply the learned model to the FULL matrix (including diagonal, though diag is always P=1)

	pdf_S_full = expon.pdf(fused_statistic_matrix, scale=1.0 / lamb)
	pdf_N_full = beta.pdf(np.clip(fused_statistic_matrix, 1e-6, 1 - 1e-6), a, b)

	numerator = pi * pdf_S_full
	denominator = numerator + (1 - pi) * pdf_N_full + 1e-10

	posterior_matrix = numerator / denominator

	return posterior_matrix, final_params


def fitGaussianMixture(fused_statistic_matrix, n_components=2, initial_params=None):
	"""
	Standard GMM fallback using Sklearn.
	We assume the component with the smaller Mean is the 'Equivalent' class.
	"""
	data = _get_off_diagonal_samples(fused_statistic_matrix).reshape(-1, 1)

	gmm = GaussianMixture(
		n_components=n_components, covariance_type="full", random_state=42
	)
	gmm.fit(data)

	# Identify which component is the "Signal" (Smallest Mean)
	means = gmm.means_.flatten()
	signal_idx = np.argmin(means)

	# Compute Posteriors for the full matrix
	N = fused_statistic_matrix.shape[0]
	reshaped_full = fused_statistic_matrix.reshape(-1, 1)
	probs = gmm.predict_proba(reshaped_full)	# Shape (N*N, 2)

	# Extract probability of the Signal Class
	posterior_flat = probs[:, signal_idx]
	posterior_matrix = posterior_flat.reshape(N, N)

	params = {
		"means": means.tolist(),
		"signal_idx": int(signal_idx),
		"weights": gmm.weights_.tolist(),
	}

	return posterior_matrix, params


def estimatePairwiseLikelihood(
	fused_statistic_matrix, method="evt", initial_params=None
):
	"""
	Orchestrator for the likelihood step.
	"""
	if method == "gmm":
		return fitGaussianMixture(fused_statistic_matrix, initial_params=initial_params)
	else:
		return fitWeibullBetaMixture(fused_statistic_matrix, initial_params=initial_params)

In [ ]:
# print("--- Test: Likelihood Estimation (EVT) on 5x5 Data ---")

# # Run the EVT Mixture Model
# posterior_evt, params_evt = estimatePairwiseLikelihood(fused, method="evt")

# print("\nFitted Parameters (likely unstable due to N=20):")
# print(params_evt)

# print("\nPosterior Probability Matrix (P(Equivalent | X)):")
# print(np.round(posterior_evt, 4))


# print("\n\n--- Test: Likelihood Estimation (GMM) on 5x5 Data ---")

# # Run the Gaussian Mixture Model
# posterior_gmm, params_gmm = estimatePairwiseLikelihood(fused, method="gmm")

# print("\nGMM Signal Index:", params_gmm["signal_idx"])
# print("Posterior Probability Matrix (GMM):")
# print(np.round(posterior_gmm, 4))

--- Test: Likelihood Estimation (EVT) on 5x5 Data ---

Fitted Parameters (likely unstable due to N=20):
{'pi': np.float64(2.1460587414674316e-06), 'lambda': np.float64(1.6512690453805945), 'alpha': np.float64(9.369808231619515), 'beta': np.float64(9.002372162148973)}

Posterior Probability Matrix (P(Equivalent | X)):
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


--- Test: Likelihood Estimation (GMM) on 5x5 Data ---

GMM Signal Index: 1
Posterior Probability Matrix (GMM):
[[1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     0.0011]
 [1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     1.    ]
 [1.     1.     1.     1.     1.    ]]


In [ ]:
def computeTransitivityPrior(likelihood_matrix):
	"""
	Calculates the Transitivity Score T_ij based on the Triangle Inequality assumption.

	T_ij represents the "Flow of Probability" from i to j through all possible intermediates k.

	Formula:
	    T_ij = (1 / N) * Sum_k ( P_ik * P_kj )
	         = (1 / N) * (Likelihood_Matrix @ Likelihood_Matrix)

	Note: We normalize by N to keep the prior in a [0, 1]-like range,
	preventing it from exploding for large graphs.

	Returns:
	    prior_matrix: Matrix of structural support scores (N x N).
	"""
	# Matrix multiplication calculates sum_k(P_ik * P_kj) for all pairs (i,j)
	# This is effectively counting expected paths of length 2
	raw_support = likelihood_matrix @ likelihood_matrix

	# Normalize by the number of possible intermediates (N) to keep scale consistent
	# Or normalize by the maximum support found to keep it relative.
	# Here we normalize by Max to treat the 'Best Connected' pair as Prior=1.0
	max_val = np.max(raw_support)
	if max_val > 0:
		prior_matrix = raw_support / max_val
	else:
		prior_matrix = raw_support	# All zeros

	return prior_matrix


def computeFinalPosterior(likelihood_matrix, prior_matrix, weight_factor=0.5):
	"""
	Synthesizes the Observation Likelihood and the Structural Prior.

	Bayesian-ish Update:
	    P(Z|X, Graph) ~ P(X|Z) * P(Z|Graph)

	Since our 'Likelihood Matrix' is technically already a posterior P(Z|X) from the mixture model,
	we are effectively fusing two expert opinions:
	1. The Feature Expert (EVT Model)
	2. The Topology Expert (Transitivity Model)

	Formula:
	    P_final = (P_evt * (P_transitivity ^ weight))

	We use geometric mean logic. If weight=0, topology is ignored.
	If weight is high, we require strong structural support to believe the signal.

	Returns:
	    final_probability_matrix: The unified probability of equivalence.
	"""
	# Add epsilon to avoid log(0) issues if we were doing log-space,
	# but for direct multiplication, 0 * anything = 0 is desired behavior (Veto power).

	# We clip the prior to [epsilon, 1.0] to ensure we don't zero out
	# a strong signal just because it has NO triangles yet (isolated pair).
	# A prior of 0.0 implies "Impossible", which is too strong.
	# We set a floor of 0.1 for the prior: "Topology is neutral/weak, rely on signal."
	safe_prior = np.clip(prior_matrix, 0.1, 1.0)

	final_posterior = likelihood_matrix * (safe_prior**weight_factor)

	return final_posterior

In [ ]:
print("--- Test: Transitivity Logic ---")

# 1. Create a synthetic Likelihood Matrix (5 nodes)
# Everything is 0.0 except our triangle
test_probs = np.zeros((5, 5))

# Strong Link 0-1
test_probs[0, 1] = 0.9
test_probs[1, 0] = 0.9

# Strong Link 1-2
test_probs[1, 2] = 0.9
test_probs[2, 1] = 0.9

# Weak Link 0-2 (The one we hope Transitivity boosts)
test_probs[0, 2] = 0.4
test_probs[2, 0] = 0.4

print("Original Likelihood (0 <-> 2):", test_probs[0, 2])

# 2. Compute Prior
prior = computeTransitivityPrior(test_probs)

print("\nComputed Prior Matrix (Support Score):")
print(np.round(prior, 4))
print(f"Prior Support for (0, 2): {prior[0, 2]:.4f}")
# Explanation: Path 0->1->2 exists (0.9*0.9 = 0.81 support).
# This should be high relative to max.

# 3. Compute Final Posterior
# We expect 0-2 to increase relative to its original weak standing,
# or at least be confirmed as valid.
final = computeFinalPosterior(test_probs, prior, weight_factor=0.5)

print(f"\nFinal Posterior (0 <-> 2): {final[0, 2]:.4f}")

# Verification:
# If prior is high (near 1.0), Final ~ 0.4 * 1.0 = 0.4
# If prior was low (0.1), Final ~ 0.4 * 0.3 = 0.12 (Suppressed)
# Since 0-2 has a strong friend (Node 1), it survives.

--- Test: Transitivity Logic ---
Original Likelihood (0 <-> 2): 0.4

Computed Prior Matrix (Support Score):
[[0.5988 0.2222 0.5    0.     0.    ]
 [0.2222 1.     0.2222 0.     0.    ]
 [0.5    0.2222 0.5988 0.     0.    ]
 [0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.    ]]
Prior Support for (0, 2): 0.5000

Final Posterior (0 <-> 2): 0.2828


In [ ]:
def generateEquivalenceProbabilities(
	model_artifacts,
	k_neighbors=50,
	modeling_method="evt",
	use_consensus_init=True,
	apply_transitivity=True,
):
	"""
	Main Execution Pipeline.

	Flow:
	    1. Local Density: Convert distances -> Local Ranks.
	    2. Fusion: Combine ranks via Minimum (EVT).
	    3. Modeling: Fit Mixture Model to estimate P(E|X).
	    4. Topology: Refine P(E|X) using Graph Transitivity.

	Args:
	    model_artifacts: Dict of prepared artifacts (dist_matrix, etc).
	    k_neighbors: Context window for local density.
	    modeling_method: "evt" or "gmm".
	    use_consensus_init: Bool, whether to use strict quantile init.
	    apply_transitivity: Bool, whether to apply structural prior.

	Returns:
	    results: Dictionary containing final matrices and params.
	"""

	# --- 1. Local Density & Fusion ---
	print(f"--- Stage 1: Computing Local Density (k={k_neighbors}) ---")
	rank_matrices = {}
	for key, artifact in model_artifacts.items():
		# Input validation
		if "dist_matrix" not in artifact:
			print(f"Skipping {key}: No dist_matrix found.")
			continue

		rank_matrices[key] = computeLocalDensityRanks(artifact["dist_matrix"], k_neighbors)

	fused_matrix = computeFusedEVTStatistic(rank_matrices)
	print(f"Fused Statistics Computed. Shape: {fused_matrix.shape}")

	# --- 2. Initialization Strategy ---
	init_params = None
	if use_consensus_init:
		print("--- Stage 2: Initializing via Consensus Heuristic ---")
		init_params = _initializeParametersViaConsensus(fused_matrix)
	else:
		print("--- Stage 2: Initializing via Naive Quantile ---")
		init_params = _initializeParametersViaNaiveQuantile(fused_matrix)

	print(f"Initial Params: {init_params}")

	# --- 3. Probabilistic Modeling ---
	print(f"--- Stage 3: Fitting Mixture Model ({modeling_method.upper()}) ---")

	likelihood_matrix, final_params = estimatePairwiseLikelihood(
		fused_matrix, method=modeling_method, initial_params=init_params
	)

	print("Model Converged.")

	# --- 4. Structural Topology ---
	posterior_matrix = likelihood_matrix
	prior_matrix = None

	if apply_transitivity:
		print("--- Stage 4: Applying Transitivity Prior ---")
		prior_matrix = computeTransitivityPrior(likelihood_matrix)
		posterior_matrix = computeFinalPosterior(likelihood_matrix, prior_matrix)
	else:
		print("--- Stage 4: Transitivity Skipped ---")

	return {
		"posterior_matrix": posterior_matrix,
		"likelihood_matrix": likelihood_matrix,	# Return raw model output too
		"prior_matrix": prior_matrix,
		"fused_stats": fused_matrix,
		"model_params": final_params,
	}

In [ ]:
# def initializeFromJaccardSet(fused_matrix, p_true_set):
# 	"""
# 	Uses the High-Precision 'Jaccard Set' to calculate exact initial parameters.

# 	Args:
# 	    fused_matrix: (N x N) The EVT statistic matrix.
# 	    p_true_set: Set of tuples {(i, j), ...} identified by findConsensusStructure.

# 	Returns:
# 	    params: Dictionary {pi, lambda, alpha, beta}
# 	"""
# 	N = fused_matrix.shape[0]

# 	# 1. Extract Signal Values (The X values for the Consensus Pairs)
# 	signal_values = []
# 	for i, j in p_true_set:
# 		signal_values.append(fused_matrix[i, j])
# 		# Also add symmetric if not present, though p_true usually has sorted tuples
# 		if (j, i) not in p_true_set:
# 			signal_values.append(fused_matrix[j, i])

# 	signal_values = np.array(signal_values)

# 	# Handle Edge Case: No consensus found
# 	if len(signal_values) == 0:
# 		print("Warning: Jaccard Set is empty. Falling back to Naive Quantile.")
# 		return _initializeParametersViaNaiveQuantile(fused_matrix)

# 	# 2. Calculate Signal Parameters (Exponential MLE)
# 	# Lambda = 1 / Mean
# 	# If the set is perfect duplicates, mean might be 0. Add epsilon.
# 	mu_s = np.mean(signal_values)
# 	lamb = 1.0 / (mu_s + 1e-6)

# 	# 3. Calculate Pi (Prior Probability of Equivalence)
# 	# Count of True Pairs / Total Pairs (N*N - N)
# 	pi = len(signal_values) / (N * N - N)

# 	# 4. Calculate Noise Parameters (Beta Method of Moments on the REST)
# 	# We sample the background by excluding the specific signal pairs is expensive,
# 	# so we just sample the whole matrix and ignore the tiny fraction of signal.
# 	# (The signal is usually < 1% of data, so it won't skew the noise calc much).
# 	data_flat = fused_matrix[~np.eye(N, dtype=bool)]
# 	mu_n = np.mean(data_flat)
# 	var_n = np.var(data_flat)

# 	# Standard Beta MOM
# 	if var_n < mu_n * (1 - mu_n):
# 		common = (mu_n * (1 - mu_n) / (var_n + 1e-9)) - 1
# 		alpha = mu_n * common
# 		beta_param = (1 - mu_n) * common
# 	else:
# 		alpha, beta_param = 1.0, 1.0

# 	params = {
# 		"pi": pi,
# 		"lambda": lamb,
# 		"alpha": max(alpha, 1.0),
# 		"beta": max(beta_param, 1.0),
# 	}

# 	return params

In [ ]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering


def extractAndPrintClusters(posterior_matrix, semantic_data, threshold=0.5):
	"""
	Converts the probabilistic output into discrete clusters and prints them.

	Logic:
	    1. Convert Probability -> Distance (Distance = 1.0 - Probability).
	    2. Apply Agglomerative Clustering with 'complete' linkage.
	       (Complete linkage ensures every member in the group has P > threshold connection to the group diameter).
	"""
	# Invert Probability to Distance
	dist_matrix = 1.0 - posterior_matrix
	# Clip for numerical safety
	dist_matrix = np.clip(dist_matrix, 0.0, 1.0)

	# Run Clustering
	# distance_threshold = 1 - P_threshold
	# If P > 0.5, then Dist < 0.5.
	model = AgglomerativeClustering(
		n_clusters=None,
		distance_threshold=(1.0 - threshold),
		metric="precomputed",
		linkage="complete",
	)

	labels = model.fit_predict(dist_matrix)

	# Organize into Groups
	groups = {}
	for idx, label in enumerate(labels):
		groups.setdefault(label, []).append(idx)

	# Filter out Singletons (Noise)
	significant_groups = [g for g in groups.values() if len(g) > 1]

	# Sort by group size (descending)
	significant_groups.sort(key=len, reverse=True)

	# Print Results
	print(f"\n{'='*80}")
	print(f"PROBABILISTIC CLUSTERING RESULTS (Threshold P > {threshold})")
	print(f"{'='*80}")
	print(f"Found {len(significant_groups)} significant groups.\n")

	for i, indices in enumerate(significant_groups):
		# Calculate 'Cluster Cohesion': Average probability of off-diagonal pairs
		sub_probs = posterior_matrix[np.ix_(indices, indices)]
		mask = ~np.eye(len(indices), dtype=bool)
		if len(indices) > 1:
			avg_prob = np.mean(sub_probs[mask])
		else:
			avg_prob = 1.0

		print(f"GROUP {i+1} (Size: {len(indices)}) [Cohesion: {avg_prob:.4f}]")

		# Print strings
		for idx in indices:
			print(f" - {semantic_data[idx]}")
		print("-" * 80)


# --- Execution Pipeline ---

# 1. Define Model Keys (Assuming data has these keys)
model_keys = ["embedding_vector", "retrieval_embedding_vector"]

print(">>> Preparing Model Artifacts...")
# prepareModelArtifacts is defined in your earlier block
artifacts = prepareModelArtifacts(qdata, model_keys, truncation_dim=256, debug=True)

# 2. Run the EVT-Topology Pipeline
print("\n>>> Running Probabilistic Pipeline...")
pipeline_results = generateEquivalenceProbabilities(
	artifacts,
	k_neighbors=50,	# Context window for Local Density
	modeling_method="evt",	# The Weibull/Beta Mixture
	use_consensus_init=True,	# Use consensus to bootstrap parameters
	apply_transitivity=True,	# Apply graph repair
)

>>> Preparing Model Artifacts...
Processing embedding_vector...
Processing retrieval_embedding_vector...

>>> Running Probabilistic Pipeline...
--- Stage 1: Computing Local Density (k=50) ---
Fused Statistics Computed. Shape: (520, 520)
--- Stage 2: Initializing via Consensus Heuristic ---
Initial Params: {'pi': 0.001, 'lambda': np.float64(49.997500124993756), 'alpha': 1.0, 'beta': 1.0}
--- Stage 3: Fitting Mixture Model (EVT) ---
Model Converged.
--- Stage 4: Applying Transitivity Prior ---


In [32]:
# 3. Print the Final Clusters
# semantic_data is the list of strings extracted in prepareModelArtifacts
semantic_data_list = artifacts[model_keys[0]]["semantic_data"]

extractAndPrintClusters(
	pipeline_results["posterior_matrix"],
	semantic_data_list,
	threshold=0.5,	# Natural probabilistic decision boundary
)


PROBABILISTIC CLUSTERING RESULTS (Threshold P > 0.5)
Found 0 significant groups.

